In [3]:
import requests, io
from PIL import Image, ImageDraw, ImageFont
import geopandas as gpd

# -----------------------------
# 1. SETTINGS
# -----------------------------
proxy = "http://127.0.0.1:8000/proxy"
layer = "Moon/EQ/LRO_WAC_Mosaic_Global_303ppd_v02/1.0.0/default/default028mm"

zoom = 3  # test with 2 or 3 first, 4 for detailed output

shapefile_path = "MoonData/MOON_nomenclature_center_pts.shp"
output_file = "moon_yolo_labeled_filtered.jpg"

min_diameter = 50   # km, only label features bigger than this
max_labels = 500    # optional: cap total number of labels to avoid clutter

# -----------------------------
# 2. FETCH + STITCH TILES
# -----------------------------
tile_size = 256
num_tiles = 2 ** zoom

print(f"Fetching {num_tiles} × {num_tiles} tiles at zoom {zoom}...")

full_img = Image.new("RGB", (num_tiles * tile_size, num_tiles * tile_size))

for x in range(num_tiles):
    for y in range(num_tiles):
        url = f"{proxy}/{layer}/{zoom}/{y}/{x}.jpg"
        r = requests.get(url)
        if r.status_code == 200:
            tile = Image.open(io.BytesIO(r.content)).convert("RGB")
            full_img.paste(tile, (x * tile_size, y * tile_size))
        else:
            print(f"⚠ Missing tile: {url}")

width, height = full_img.size
print(f"✅ Stitched image size: {width} × {height}")

# -----------------------------
# 3. LOAD SHAPEFILE + FILTER
# -----------------------------
gdf = gpd.read_file(shapefile_path)
print("Columns:", gdf.columns)
print("Total features before filtering:", len(gdf))

# Filter by diameter (if column exists)
if "diameter" in gdf.columns:
    gdf = gdf[gdf["diameter"].astype(float) > min_diameter]

# Optionally keep only the top N largest
if "diameter" in gdf.columns:
    gdf = gdf.sort_values("diameter", ascending=False).head(max_labels)

print("Total features after filtering:", len(gdf))

# -----------------------------
# 4. LON/LAT → PIXEL
# -----------------------------
def lonlat_to_pixel(lon, lat, width, height):
    # Convert 0–360 longitudes to -180–180
    if lon > 180:
        lon -= 360

    x = (lon + 180.0) / 360.0 * width
    y = (90.0 - lat) / 180.0 * height
    return int(x), int(y)
# -----------------------------
# 5. DRAW LABELS
# -----------------------------
draw = ImageDraw.Draw(full_img)
font = ImageFont.load_default()

for idx, row in gdf.iterrows():
    try:
        name = row["name"]
        ftype = row.get("type", "")
        lon, lat = row["center_lon"], row["center_lat"]

        px, py = lonlat_to_pixel(lon, lat, width, height)

        # Draw red dot / small box
        r = 2
        draw.rectangle([px-r, py-r, px+r, py+r], outline="red", width=1)

        # Label text
        label = f"{name} ({ftype})" if ftype else name
        draw.text((px+5, py), label, fill="yellow", font=font)
    except Exception as e:
        print(f"Skipping row {idx} due to error: {e}")

# -----------------------------
# 6. SAVE
# -----------------------------
full_img.save(output_file)
print(f"✅ Saved filtered labeled Moon map → {output_file}")

Fetching 8 × 8 tiles at zoom 3...
✅ Stitched image size: 2048 × 2048
Columns: Index(['name', 'clean_name', 'approvaldt', 'origin', 'diameter', 'center_lon',
       'center_lat', 'type', 'code', 'approval', 'min_lon', 'max_lon',
       'min_lat', 'max_lat', 'ethnicity', 'continent', 'quad_name',
       'quad_code', 'link', 'geometry'],
      dtype='object')
Total features before filtering: 9084
Total features after filtering: 500
✅ Saved filtered labeled Moon map → moon_yolo_labeled_filtered.jpg


In [5]:
import requests, io
from PIL import Image, ImageDraw, ImageFont
import geopandas as gpd

# -----------------------------
# 1. SETTINGS
# -----------------------------
proxy = "http://127.0.0.1:8000/proxy"
layer = "Mars/EQ/Mars_Viking_MDIM21_ClrMosaic_global_232m/1.0.0/default/default028mm"

zoom = 3  # test with 2 or 3 first, 4 for detailed output

shapefile_path = "MarsData/MARS_nomenclature_center_pts.shp"
output_file = "mars_yolo_labeled_filtered.jpg"

min_diameter = 50   # km, only label features bigger than this
max_labels = 500    # optional: cap total number of labels to avoid clutter

# -----------------------------
# 2. FETCH + STITCH TILES
# -----------------------------
tile_size = 256
num_tiles = 2 ** zoom

print(f"Fetching {num_tiles} × {num_tiles} tiles at zoom {zoom}...")

full_img = Image.new("RGB", (num_tiles * tile_size, num_tiles * tile_size))

for x in range(num_tiles):
    for y in range(num_tiles):
        url = f"{proxy}/{layer}/{zoom}/{y}/{x}.jpg"
        r = requests.get(url)
        if r.status_code == 200:
            tile = Image.open(io.BytesIO(r.content)).convert("RGB")
            full_img.paste(tile, (x * tile_size, y * tile_size))
        else:
            print(f"⚠️ Missing tile: {url}")

width, height = full_img.size
print(f"✅ Stitched image size: {width} × {height}")

# -----------------------------
# 3. LOAD SHAPEFILE + FILTER
# -----------------------------
gdf = gpd.read_file(shapefile_path)
print("Columns:", gdf.columns)
print("Total features before filtering:", len(gdf))

# Filter by diameter (if column exists)
if "diameter" in gdf.columns:
    gdf = gdf[gdf["diameter"].astype(float) > min_diameter]

# Optionally keep only the top N largest
if "diameter" in gdf.columns:
    gdf = gdf.sort_values("diameter", ascending=False).head(max_labels)

print("Total features after filtering:", len(gdf))

# -----------------------------
# 4. LON/LAT → PIXEL
# -----------------------------
def lonlat_to_pixel(lon, lat, width, height):
    # Convert 0–360 longitudes to -180–180
    if lon > 180:
        lon -= 360

    x = (lon + 180.0) / 360.0 * width
    y = (90.0 - lat) / 180.0 * height
    return int(x), int(y)
# -----------------------------
# 5. DRAW LABELS
# -----------------------------
draw = ImageDraw.Draw(full_img)
font = ImageFont.load_default()
for idx, row in gdf.iterrows():
    try:
        name = row["name"]
        ftype = row.get("type", "")
        lon, lat = row["center_lon"], row["center_lat"]

        px, py = lonlat_to_pixel(lon, lat, width, height)

        # Draw red dot / small box
        r = 2
        draw.rectangle([px-r, py-r, px+r, py+r], outline="red", width=1)

        # Label text
        label = f"{name} ({ftype})" if ftype else name
        draw.text((px+5, py), label, fill="yellow", font=font)
    except Exception as e:
        print(f"Skipping row {idx} due to error: {e}")

# -----------------------------
# 6. SAVE
# -----------------------------
full_img.save(output_file)
print(f"✅ Saved filtered labeled Mars map → {output_file}")

Fetching 8 × 8 tiles at zoom 3...
✅ Stitched image size: 2048 × 2048
Columns: Index(['name', 'clean_name', 'approvaldt', 'origin', 'diameter', 'center_lon',
       'center_lat', 'type', 'code', 'approval', 'min_lon', 'max_lon',
       'min_lat', 'max_lat', 'ethnicity', 'continent', 'quad_name',
       'quad_code', 'link', 'geometry'],
      dtype='object')
Total features before filtering: 2036
Total features after filtering: 500
✅ Saved filtered labeled Mars map → mars_yolo_labeled_filtered.jpg


In [ ]:
import requests, io
from PIL import Image, ImageDraw, ImageFont
import geopandas as gpd

# -----------------------------
# 1. SETTINGS
# -----------------------------
proxy = "http://127.0.0.1:8000/proxy"
layer = "Mercury/EQ/Mercury_MESSENGER_MDIS_Basemap_BDR_Mosaic_Global_166m/1.0.0/default/default028mm"

zoom = 3  # test with 2 or 3 first, 4 for detailed output

shapefile_path = "MercuryData/MERCURY_nomenclature_center_pts.shp"
output_file = "mercury_yolo_labeled_filtered.jpg"

min_diameter = 50   # km, only label features bigger than this
max_labels = 500    # optional: cap total number of labels to avoid clutter

# -----------------------------
# 2. FETCH + STITCH TILES
# -----------------------------
tile_size = 256
num_tiles = 2 ** zoom

print(f"Fetching {num_tiles} × {num_tiles} tiles at zoom {zoom}...")

full_img = Image.new("RGB", (num_tiles * tile_size, num_tiles * tile_size))

for x in range(num_tiles):
    for y in range(num_tiles):
        url = f"{proxy}/{layer}/{zoom}/{y}/{x}.jpg"
        r = requests.get(url)
        if r.status_code == 200:
            tile = Image.open(io.BytesIO(r.content)).convert("RGB")
            full_img.paste(tile, (x * tile_size, y * tile_size))
        else:
            print(f"⚠ Missing tile: {url}")

width, height = full_img.size
print(f"✅ Stitched image size: {width} × {height}")

# -----------------------------
# 3. LOAD SHAPEFILE + FILTER
# -----------------------------
gdf = gpd.read_file(shapefile_path)
print("Columns:", gdf.columns)
print("Total features before filtering:", len(gdf))

# Filter by diameter (if column exists)
if "diameter" in gdf.columns:
    gdf = gdf[gdf["diameter"].astype(float) > min_diameter]

# Optionally keep only the top N largest
if "diameter" in gdf.columns:
    gdf = gdf.sort_values("diameter", ascending=False).head(max_labels)

print("Total features after filtering:", len(gdf))

# -----------------------------
# 4. LON/LAT → PIXEL
# -----------------------------
def lonlat_to_pixel(lon, lat, width, height):
    # Convert 0–360 longitudes to -180–180
    if lon > 180:
        lon -= 360

    x = (lon + 180.0) / 360.0 * width
    y = (90.0 - lat) / 180.0 * height
    return int(x), int(y)
# -----------------------------
# 5. DRAW LABELS
# -----------------------------
draw = ImageDraw.Draw(full_img)
font = ImageFont.load_default()

for idx, row in gdf.iterrows():
    try:
        name = row["name"]
        ftype = row.get("type", "")
        lon, lat = row["center_lon"], row["center_lat"]

        px, py = lonlat_to_pixel(lon, lat, width, height)

        # Draw red dot / small box
        r = 2
        draw.rectangle([px-r, py-r, px+r, py+r], outline="red", width=1)

        # Label text
        label = f"{name} ({ftype})" if ftype else name
        draw.text((px+5, py), label, fill="yellow", font=font)
    except Exception as e:
        print(f"Skipping row {idx} due to error: {e}")

# -----------------------------
# 6. SAVE
# -----------------------------
full_img.save(output_file)
print(f"✅ Saved filtered labeled Mercury map → {output_file}")

Fetching 8 × 8 tiles at zoom 3...


In [ ]:
import nbformat
import numpy as np
import plotly.graph_objects as go

# Convert basemap (keep grayscale for now)
gray_img = np.array(full_img.convert("L"))
h, w = gray_img.shape

# Use same resolution as image
phi = np.linspace(0, np.pi, h)        # rows
theta = np.linspace(0, 2*np.pi, w)    # columns
theta, phi = np.meshgrid(theta, phi)

# Sphere coordinates
x = np.cos(theta) * np.sin(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(phi)

# Directly use grayscale image as surfacecolor
surface = go.Surface(
    x=x, y=y, z=z,
    surfacecolor=gray_img,   # every pixel mapped
    colorscale="gray",
    cmin=0, cmax=255,
    showscale=False,
    lighting=dict(
        ambient=0.6,
        diffuse=0.8,
        specular=0.5,
        roughness=0.4,
        fresnel=0.05
    ),
    lightposition=dict(x=100, y=200, z=0)
)

# Add labels as before
labels = []
for idx, row in gdf.iterrows():
    lon, lat = row["center_lon"], row["center_lat"]
    if lon > 180:
        lon -= 360
    lon_rad = np.deg2rad(lon)
    lat_rad = np.deg2rad(lat)

    lx = np.cos(lon_rad) * np.cos(lat_rad)
    ly = np.sin(lon_rad) * np.cos(lat_rad)
    lz = np.sin(lat_rad)

    labels.append(go.Scatter3d(
        x=[lx], y=[ly], z=[lz],
        mode="markers+text",
        marker=dict(size=3, color="red"),
        text=[row["name"]],
        textposition="top center",
        textfont=dict(size=10, color="yellow")
    ))

# Build figure
fig = go.Figure(data=[surface] + labels)
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode="data",
        bgcolor="black"
    ),
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()

5.10.4
